In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
cc_df = pd.read_csv('fraud.csv')

In [ ]:
pd.set_option('display.max_columns', 50)

In [ ]:
cc_df.head()

In [ ]:
cc_df.info()

In [ ]:
cc_df.describe()

In [ ]:
cc_df['merchant'].unique()

In [ ]:
cc_df['job'].unique()

In [ ]:
cc_df['cc_num'].nunique()

In [ ]:
cc_df.drop(['merchant','first','last','street','city','state','zip','job','trans_num','unix_time'], axis = 1, inplace= True)
cc_df.head(5)

In [ ]:
# cc_num 컬럼의 값을 기준으로 정렬합니다.
cc_df.sort_values('cc_num')

In [ ]:
# z-score 계산을 위한 실습용 데이터입니다.

temp = pd.DataFrame({'a': [10,20,30,20,10,200], 'b': [100,300,200,150,250,200], 'c': [10, 500, 20, 250, 25, 200]})

In [ ]:
# temp 데이터의 컬럼별 평균을 계산합니다.
temp.mean()

In [ ]:
temp.std()

In [ ]:
# 각각의 데이터에 대해 a 컬럼의 z-score를 계산합니다.
# (data - mean) / std
(temp['a'] - 48.33) / 74.67

In [ ]:
# Q. 각각의 데이터에 대해 b 컬럼의 z-score를 계산합니다.
(temp['b'] - 200.00) /  70.71

In [ ]:
# Q. 각각의 데이터에 대해 c 컬럼의 z-score를 계산합니다.
(temp['c'] - 167.50) / 192.50

In [ ]:
# cc_num 컬럼의 값마다 데이터 개수를 계산합니다.
cc_df['cc_num'].value_counts()

In [ ]:
# cc_num 컬럼의 값마다 amt의 평균, 표준편차를 계산하여 amt_info에 저장합니다.
amt_info = cc_df.groupby('cc_num')['amt'].agg(['mean','std']).reset_index()

In [ ]:
amt_info.to_pickle('./amt_info.pkl')

In [ ]:
# Q. cc_num 컬럼을 기준으로, cc_df와 amt_info 데이터를 합쳐서 cc_df에 저장합니다.
# (left merge를 수행합니다.)

cc_df = cc_df.merge(amt_info, how='left')
cc_df.head(5)

In [ ]:
# Q. 결제금액(amt)의 z-score를 계산하여 amt_z 컬럼에 저장합니다.
# 평균: mean, 표준편차: std

cc_df['amt_z'] = (cc_df['amt'] - cc_df['mean']) / cc_df['std']

In [ ]:
# 사기 거래 데이터(is_fraud=1)를 확인합니다.

cc_df[cc_df['is_fraud'] == 1]

In [ ]:
# z-score 계산이 완료되었으니 mean, std 컬럼을 제거합니다.

cc_df.drop(['mean','std'], axis = 1, inplace = True)
cc_df.head(5)

In [ ]:
# cc_num, category 컬럼의 값마다 결제금액(amt)의 평균, 표준편차를 계산하여 cat_info에 저장합니다.

cat_info = cc_df.groupby(['cc_num','category'])['amt'].agg(['mean','std']).reset_index()

In [ ]:
cat_info.to_pickle('./cat_info.pkl')

In [ ]:
cc_df = cc_df.merge(cat_info, how ='left', on='cc_num')
cc_df.head(5)

In [ ]:
# Q. 결제금액(amt)의 z-score를 계산하여 cat_amt_z 컬럼에 저장합니다.
# 평균: mean, 표준편차: std
cc_df['cat_amt_z'] = (cc_df['amt'] - cc_df['mean']) / cc_df['std']

In [ ]:
# z-score 계산이 완료되었으니 mean, std 컬럼을 제거합니다.

cc_df.drop(['mean','std'], axis =1 , inplace = True)

In [ ]:
cc_df.head()

In [ ]:
# Q. trans_date_trans_time 컬럼에서 시간 값을 추출하여 hour 컬럼에 저장합니다.
# (힌트: datetime 자료형을 이용합니다.)

cc_df['trans_date_trans_time'] = pd.to_datetime(cc_df['trans_date_trans_time'])
cc_df['hour'] = cc_df['trans_date_trans_time'].dt.hour

In [ ]:
# 결제시간을 morning, afternoon, night, evening으로 분류하기 위해 함수를 정의합니다.

def hour_func(x):
    if (x >= 6) & (x < 12):
        return 'morning'
    elif (x >= 12) & (x < 18):
        return 'afternoon'
    elif (x >= 18) & (x < 23):
        return 'night'
    else:
        return 'evening'

In [ ]:
# Q. hour 컬럼에 hour_func 함수를 적용한 값을 hour_cat 컬럼에 저장합니다.

cc_df['hour_cat'] = cc_df['hour'].apply(hour_func)

In [ ]:
# Q. 시간대별로 데이터 수를 계산하여 출력합니다.

cc_df['hour'].value_counts()

In [ ]:
# cc_num 컬럼의 값마다 amt 컬럼의 데이터 수를 계산하여 all_cnt 변수로 저장합니다.

all_cnt = cc_df.groupby('cc_num')['amt'].count().reset_index()

In [ ]:
# cc_num, hour_cat 컬럼의 값마다 amt 컬럼의 데이터 수를 계산하여 hour_cnt 변수로 저장합니다.

hour_cnt = cc_df.groupby(['cc_num','hour_cat'])['amt'].count().reset_index()

In [ ]:
all_cnt.head()

In [ ]:
hour_cnt.head()

In [ ]:
# Q. cc_num 컬럼을 기준으로 hour_cnt와 all_cnt를 합칩니다.
# (hour_cnt에서 left merge를 수행합니다.)

hour_cnt = hour_cnt.merge(all_cnt, on = 'cc_num' , how='left')

In [ ]:
hour_cnt.head()

In [ ]:
# Q. amt_x와 amt_y 컬럼의 이름을 각각 hour_cnt, total_cnt로 변경합니다.

hour_cnt = hour_cnt.rename(columns= {'amt_x' : 'hour_cnt','amt_y':'total_cnt' })

In [ ]:
hour_cnt.head()

In [ ]:
# Q. 전체 거래 건수(total_cnt)에 대한 시간대별 거래 건수(hour_cnt)의 비율을 계산하여
# hour_perc 컬럼에 저장합니다.
hour_cnt['hour_perc'] = hour_cnt['total_cnt'] / hour_cnt['hour_cnt']

In [ ]:
hour_cnt = hour_cnt[['cc_num','hour_cat','hour_perc']]

In [ ]:
hour_cnt.to_pickle('./hour_cnt.pkl')

In [ ]:
# Q. cc_num, hour_cat 컬럼을 기준으로 cc_df와 hour_cnt 데이터를 합칩니다.
# (cc_df에서 left merge를 수행합니다.)

cc_df.merge(hour_cnt, on= ['cc_num', 'hour_cat'], how='left')

In [ ]:
cc_df.head()

In [ ]:
# 시간 관련 feature 분석이 완료되었으므로 불필요해진 컬럼들을 제거합니다.

cc_df.drop(['trans_date_trans_time', 'hour', 'hour_cat'], axis =1 , inplace = True)
cc_df.head(10)

In [ ]:
!pip install geopy

In [ ]:
from geopy.distance import distance

In [ ]:
# 이렇게 위도와 경도 값으로 두 지점 사이의 거리를 구할 수 있습니다.

distance((48.8878, -118.2105), (49.159047, -118.186462)).km

In [ ]:
# 고객의 위치와 상점의 위치 사이의 거리를 계산하여 distance 컬럼에 저장합니다.
# (모든 데이터가 계산되므로 실행 시간이 오래 걸릴 수 있습니다.)

cc_df['distance'] = cc_df.apply(lambda x: distance((x['lat'], x['long']), (x['merch_lat'], x['merch_long'])).km, axis = 1)

In [ ]:
from datetime import datetime

In [ ]:
# 데이터 10000개만 계산해도 시간이 이렇게 걸립니다.

start_time =  datetime.now()
cc_df.head(10000).apply(lambda x: distance((x['lat'], x['long']), (x['merch_lat'], x['merch_long'])).km, axis = 1)
datetime.now() - start_time

In [ ]:
cc_df.head()

In [ ]:
# Q. cc_num 컬럼의 값마다 distance의 평균, 표준편차를 계산하여 dist_info 변수에 저장합니다.  qq

dist_info = cc_df.groupby('cc_num')['distance'],agg(['mean','std']).reset_index()

In [ ]:
dist_info.to_pickle('./dist_info.pkl')

In [ ]:
# Q. cc_num 컬럼을 기준으로 cc_df와 dist_info 데이터를 합칩니다.
# (cc_df에서 left merge를 수행합니다.)

cc_df = cc_df.merge(dist_info, on = 'cc_num', how='left')

In [ ]:
# Q. distance 값의 z-score를 계산하여 dist_z 컬럼에 저장합니다.

cc_df['dist_z'] = (cc_df['distance'] - cc_df['mean']) / cc_df['std']

In [ ]:
# 불필요해진 컬럼들을 제거합니다.

cc_df.drop(['lat','long','merch_lat','merch_long','mean','std'], axis = 1, inplace = True)

In [ ]:
cc_df

In [ ]:
# Q. dob 컬럼에서 연도 값만 추출하여 dob 컬럼에 저장합니다.

cc_df['dob'] = pd.to_datetime(cc_df['dob']).dt.year

In [ ]:
# 범주형 컬럼 'category'에 몇 종류의 값이 있는지 확인해봅시다.

cc_df['category'].nunique()

In [ ]:
# Q. cc_df의 범주형 데이터에 원-핫 인코딩을 적용합니다.
# (drop_first 옵션은 True로 설정합니다.)

cc_df = pd.get_dummies(cc_df, drop_first = True)

In [ ]:
cc_df.head()

In [ ]:
# 이제 불필요해진 cc_num 컬럼을 제거합니다.

cc_df.drop('cc_num', axis = 1, inplace = True)